In [1]:
#Search Data Scientist Jobs in NY
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
def getInfo(postings_desc, postings_link):    
    title = []
    is_new = []
    link = []
    company = []
    rating = []
    location = []
    post_date = []

    for post in postings_desc:
        elem = post.find_all("span")
        if elem[0].text=="new":
            is_new.append("Y")
            title.append(elem[1].text)
        else:
            is_new.append("N")
            title.append(elem[0].text)
            
        company.append(post.find("span", attrs={"class", "companyName"}).text)

        if post.find("span", attrs={"class", "ratingNumber"}) == None:
            rating.append("")
        else:
            rating.append(post.find("span", attrs={"class", "ratingNumber"}).text)

        location.append(post.find("div", attrs={"class", "companyLocation"}).text)

        post_date.append(post.find("span", attrs={"class", "date"}).text.replace("Posted", "").replace("EmployerActive ", ""))

    # for post in postings_link:
    #     link.append(post["href"].replace("/rc/clk", "indeed.com/viewjob"))

    post_date_num = []
    for date in post_date:
        if date == "Today":
            post_date_num.append(0)
        elif date == "Just posted":
            post_date_num.append(-1)
        elif date == "Hiring ongoing":
            post_date_num.append(-2)
        else:
            post_date_num.append(int(date[:2].replace(" ", "")))

    job_postings = pd.DataFrame(title, columns=["Title"])
    job_postings["Company"] = company
    job_postings["Rating"] = rating
    job_postings["Location"] = location
    job_postings["Is New?"] = is_new
    job_postings["Aging"] = post_date
    job_postings["Sort_Key"] = post_date_num
    # job_postings["Link"] = link

    return job_postings

In [3]:
def website_query(url, short_url, save_name):  
    current_page = 1

    while True:
        if current_page>1:
            if soup.find("a", {"aria-label":str(current_page)}) is None:
                break
            else:
                url=short_url+soup.find("a", {"aria-label":str(current_page)})["href"]

        response = requests.get(url)
        soup = BeautifulSoup(response.text,features="html.parser")

        postings_link = soup.find_all("a", {"class":"jcs-JobTitle"})
        postings_desc = soup.find_all("div", {"class":"job_seen_beacon"})

        if current_page == 1:
            complete_postings = getInfo(postings_desc, postings_link)
        else:
            complete_postings = pd.concat([complete_postings, getInfo(postings_desc, postings_link)], ignore_index=True)

        current_page+=1



    complete_postings.sort_values(by=['Sort_Key'], inplace=True)
    complete_postings.drop("Sort_Key", axis=1, inplace=True)

    complete_postings = complete_postings.loc[(complete_postings.Aging != "30+ days ago") & (complete_postings.Rating != "")]

    complete_postings.reset_index(drop=True, inplace=True)

    complete_postings.to_excel("Job Postings_"+save_name+".xlsx", index=False)

In [4]:
website_query("https://www.indeed.com/jobs?q=data%20scientist&l=New%20York%2C%20NY", 
    "https://www.indeed.com", "Data Scientist_NY")

website_query("https://uk.indeed.com/jobs?q=data%20scientist&l=London%2C%20Greater%20London", 
    "https://uk.indeed.com", "Data Scientist_London")